In [8]:
import tensorflow as tf
import sys

# change this as you see fit
image_path = "fox.jpg"

# Read in the image_data
image_data = tf.gfile.FastGFile(image_path, 'rb').read()
#if opencv
#import cv2
#image_data = cv2.imread(image_path)

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line in tf.gfile.GFile("retrained_labels.txt")]
print(label_lines)

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
    g = tf.get_default_graph()
    #print([op.name for op in g.get_operations()])

with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
    
    print(predictions.shape)
    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    print(top_k)
    
    for node_id in top_k:
        print(node_id)
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        print('%s (score = %.5f)' % (human_string, score))

['iu', 'smoking', 'suji']
(1, 3)
[1 2 0]
1
smoking (score = 0.92020)
2
suji (score = 0.05880)
0
iu (score = 0.02101)
